In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import binom, hypergeom
from sklearn.linear_model import LogisticRegression

Data from http://users.nber.org/%7Erdehejia/nswdata2.html

The variables from left to right are: treatment indicator (1 if treated, 0 if not treated), age, education, Black (1 if black, 0 otherwise), Hispanic (1 if Hispanic, 0 otherwise), married (1 if married, 0 otherwise), nodegree (1 if no degree, 0 otherwise), RE74 (earnings in 1974), RE75 (earnings in 1975), and RE78 (earnings in 1978).

In [2]:
col_names = ['treated', 'age', 'education', 'black', 'hispanic', 'married', 'nodegree', 're74', 're75', 're78']
treated = pd.read_table('data/nswre74_treated.txt',sep='\s+', header=None, names=col_names)
control = pd.read_table('data/nswre74_control.txt',sep='\s+', header=None, names=col_names)
data = pd.concat([treated, control])

In [3]:
lor = LogisticRegression()
propensity = lor.fit(data[col_names[1:]], data.treated)
pscore = propensity.predict_proba(data[col_names[1:]])[:,1]

data['prop_score'] = pscore

In [4]:
data.head()

,treated,age,education,black,hispanic,married,nodegree,re74,re75,re78,prop_score
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,0.0,9930.0460,0.472255
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,0.0,3595.8940,0.240599
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,24909.4500,0.723242
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,7506.1460,0.399273
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,0.0,289.7899,0.363913


### One to One matching without replacement

In [5]:
np.random.seed(10)

def Match(groups, pscore, caliper=0.05):
    '''
    Inputs: 
    groups = indicator to identify control and study group. Must be two groups. Number of treated (study) records should be
            smaller than untreated (control) records.
    pscore = propensity scores for each observation. Groups and pscore must be in the same order ( same indices).
    caliper = maximum difference in matched propensity scores.
    
    '''
    if any(pscore <= 0) or any(pscore >= 1):
        raise ValueError('Propensity scores must be between 0 and 1')
    elif not(0<caliper<1):
        raise ValueError('Caliper must be between 0 and 1')
    elif len(groups) != len(pscore):
        raise ValueError('groups and pscore must have same dimension')
    elif len(groups.unique()) != 2:
        raise ValueError('wrong number of groups')
     
    
    groups = groups == groups.unique()[0]
    N = len(groups)
    N1 = groups.sum(); 
    N2 = N - N1;
    g1, g2 = (pscore[groups == 1]),(pscore[groups == 0])
    # N1 is study, N2 is control
    # assuming number of study groups is less than control groups
    if N1 > N2:
        N1, N2, g1, g2 = N2, N1, g2, g1

    if (N1 != len(g1)) or (N2 != len(g2)):
        raise ValueError('Number of study records must be less than control records')

    morder = np.random.permutation(N1)
    matches = pd.Series(np.empty(N1))
    matches[:] = np.NAN

    for m in morder:
        dist = abs(g1[m] - g2)
        if dist.min() <= caliper:
            matches[m] = dist.idxmin()
            g2 = g2.drop(matches[m])

    return matches
        
        
        

In [6]:
matches = Match(data.treated, data.prop_score)

In [7]:
g1, g2 = data.prop_score[data.treated == 1], data.prop_score[data.treated == 0]

In [26]:
study_grp, control_grp = data[data.treated == 1], data[data.treated == 0]


In [18]:
study = pd.DataFrame(g1).reset_index()
study.columns = ['std_indx', 'std_pscore']
control = pd.DataFrame(g2[matches]).reset_index()
control.columns = ['ctl_indx', 'ctl_pscore']


In [19]:
final_df = study.join(control)
final_df.head()

,std_indx,std_pscore,ctl_indx,ctl_pscore
0,0,0.472255,216.0,0.471468
1,1,0.240599,84.0,0.241766
2,2,0.723242,207.0,0.687753
3,3,0.399273,215.0,0.399432
4,4,0.363913,115.0,0.363703


### Records from study group matched with records from control group. With index we can see with study record matched with which control record ( from study_grp and control_grp).